# Global settings

In [ ]:
# References:
# This source code file refers to:
# https://colab.research.google.com/github/dpressel/dlss-tutorial/blob/master/1_pretrained_vectors.ipynb
# https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f


In [1]:
DATASET_NAME = 'qemu'
MODEL_NAME = 'cnn'

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Tuple
import io
import re
import codecs
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset
import random
import pandas as pd
import shutil

def write_to_file(text, path, mode='a'): # 'a': append; 'w': overwrite
    with open(path, mode) as f:
        f.write(text)

def mkdir_if_not_exist(directory):
    if not directory: return
    if not os.path.exists(directory):
        os.mkdir(directory)

def remove_file_if_exist(path):
    if not path: return
    if os.path.exists(path):
        try:
            os.remove(path)
        except:
            shutil.rmtree(path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('using', device)

# The following randomization refers to: https://github.com/ICL-ml4csec/VulBERTa/blob/main/Finetuning_VulBERTa-MLP.ipynb
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
os.environ['WANDB_DISABLED'] = 'true'
os.environ['WANDB_MODE'] = 'dryrun'


using cuda


# Models for classification

### CNN

In [3]:
class ParallelConv(nn.Module):

    def __init__(self, input_dims, filters, dropout=0.5):
        super().__init__()
        convs = []        
        self.output_dims = sum([t[1] for t in filters])
        for (filter_length, output_dims) in filters:
            pad = filter_length//2
            conv = nn.Sequential(
                nn.Conv1d(input_dims, output_dims, filter_length, padding=pad),
                nn.ReLU()
            )
            convs.append(conv)
        # Add the module so its managed correctly
        self.convs = nn.ModuleList(convs)
        self.conv_drop = nn.Dropout(dropout)


    def forward(self, input_bct):
        mots = []
        for conv in self.convs:
            # In Conv1d, data BxCxT, max over time
            conv_out = conv(input_bct)
            mot, _ = conv_out.max(2)
            mots.append(mot)
        mots = torch.cat(mots, 1)
        return self.conv_drop(mots)

class ConvClassifier(nn.Module):

    def __init__(self, embeddings, num_classes, embed_dims,
                 filters=[(2, 100), (3, 100), (4, 100)],
                 dropout=0.5, hidden_units=[]):
        super().__init__()
        self.embeddings = embeddings
        self.dropout = nn.Dropout(dropout)
        self.convs = ParallelConv(embed_dims, filters, dropout)
        
        input_units = self.convs.output_dims
        output_units = self.convs.output_dims
        sequence = []
        for h in hidden_units:
            sequence.append(self.dropout(nn.Linear(input_units, h)))
            input_units = h
            output_units = h
            
        sequence.append(nn.Linear(output_units, num_classes))
        self.outputs = nn.Sequential(*sequence)

    def forward(self, inputs):
        one_hots, lengths = inputs
        embed = self.dropout(self.embeddings(one_hots))
        embed = embed.transpose(1, 2).contiguous()
        hidden = self.convs(embed)
        linear = self.outputs(hidden)
        return F.log_softmax(linear, dim=-1)


# Utils

In [5]:

class ConfusionMatrix:
    """Confusion matrix with metrics

    This class accumulates classification output, and tracks it in a confusion matrix.
    Metrics are available that use the confusion matrix
    """
    def __init__(self, labels):
        """Constructor with input labels

        :param labels: Either a dictionary (`k=int,v=str`) or an array of labels
        """
        if type(labels) is dict:
            self.labels = []
            for i in range(len(labels)):
                self.labels.append(labels[i])
        else:
            self.labels = labels
        nc = len(self.labels)
        self._cm = np.zeros((nc, nc), dtype=np.int)

    def add(self, truth, guess):
        """Add a single value to the confusion matrix based off `truth` and `guess`

        :param truth: The real `y` value (or ground truth label)
        :param guess: The guess for `y` value (or assertion)
        """

        self._cm[truth, guess] += 1

    def __str__(self):
        values = []
        width = max(8, max(len(x) for x in self.labels) + 1)
        for i, label in enumerate([''] + self.labels):
            values += ["{:>{width}}".format(label, width=width+1)]
        values += ['\n']
        for i, label in enumerate(self.labels):
            values += ["{:>{width}}".format(label, width=width+1)]
            for j in range(len(self.labels)):
                values += ["{:{width}d}".format(self._cm[i, j], width=width + 1)]
            values += ['\n']
        values += ['\n']
        return ''.join(values)

    def save(self, outfile):
        ordered_fieldnames = OrderedDict([("labels", None)] + [(l, None) for l in self.labels])
        with open(outfile, 'w') as f:
            dw = csv.DictWriter(f, delimiter=',', fieldnames=ordered_fieldnames)
            dw.writeheader()
            for index, row in enumerate(self._cm):
                row_dict = {l: row[i] for i, l in enumerate(self.labels)}
                row_dict.update({"labels": self.labels[index]})
                dw.writerow(row_dict)

    def reset(self):
        """Reset the matrix
        """
        self._cm *= 0

    def get_correct(self):
        """Get the diagonals of the confusion matrix

        :return: (``int``) Number of correct classifications
        """
        return self._cm.diagonal().sum()

    def get_total(self):
        """Get total classifications

        :return: (``int``) total classifications
        """
        return self._cm.sum()

    def get_acc(self):
        """Get the accuracy

        :return: (``float``) accuracy
        """
        return float(self.get_correct())/self.get_total()

    def get_recall(self):
        """Get the recall

        :return: (``float``) recall
        """
        total = np.sum(self._cm, axis=1)
        total = (total == 0) + total
        return np.diag(self._cm) / total.astype(float)

    def get_support(self):
        return np.sum(self._cm, axis=1)

    def get_precision(self):
        """Get the precision
        :return: (``float``) precision
        """

        total = np.sum(self._cm, axis=0)
        total = (total == 0) + total
        return np.diag(self._cm) / total.astype(float)

    def get_mean_precision(self):
        """Get the mean precision across labels

        :return: (``float``) mean precision
        """
        return np.mean(self.get_precision())

    def get_weighted_precision(self):
        return np.sum(self.get_precision() * self.get_support())/float(self.get_total())

    def get_mean_recall(self):
        """Get the mean recall across labels

        :return: (``float``) mean recall
        """
        return np.mean(self.get_recall())

    def get_weighted_recall(self):
        return np.sum(self.get_recall() * self.get_support())/float(self.get_total())

    def get_weighted_f(self, beta=1):
        return np.sum(self.get_class_f(beta) * self.get_support())/float(self.get_total())

    def get_macro_f(self, beta=1):
        """Get the macro F_b, with adjustable beta (defaulting to F1)

        :param beta: (``float``) defaults to 1 (F1)
        :return: (``float``) macro F_b
        """
        if beta < 0:
            raise Exception('Beta must be greater than 0')
        return np.mean(self.get_class_f(beta))

    def get_class_f(self, beta=1):
        p = self.get_precision()
        r = self.get_recall()

        b = beta*beta
        d = (b * p + r)
        d = (d == 0) + d

        return (b + 1) * p * r / d

    def get_f(self, beta=1):
        """Get 2 class F_b, with adjustable beta (defaulting to F1)

        :param beta: (``float``) defaults to 1 (F1)
        :return: (``float``) 2-class F_b
        """
        p = self.get_precision()[1]
        r = self.get_recall()[1]
        if beta < 0:
            raise Exception('Beta must be greater than 0')
        d = (beta*beta * p + r)
        if d == 0:
            return 0
        return (beta*beta + 1) * p * r / d

    def get_all_metrics(self):
        """Make a map of metrics suitable for reporting, keyed by metric name

        :return: (``dict``) Map of metrics keyed by metric names
        """
        metrics = {'acc': self.get_acc()}
        # If 2 class, assume second class is positive AKA 1
        if len(self.labels) == 2:
            metrics['precision'] = self.get_precision()[1]
            metrics['recall'] = self.get_recall()[1]
            metrics['f1'] = self.get_f(1)
        else:
            metrics['mean_precision'] = self.get_mean_precision()
            metrics['mean_recall'] = self.get_mean_recall()
            metrics['macro_f1'] = self.get_macro_f(1)
            metrics['weighted_precision'] = self.get_weighted_precision()
            metrics['weighted_recall'] = self.get_weighted_recall()
            metrics['weighted_f1'] = self.get_weighted_f(1)
        return metrics

    def add_batch(self, truth, guess):
        """Add a batch of data to the confusion matrix

        :param truth: The truth tensor
        :param guess: The guess tensor
        :return:
        """
        for truth_i, guess_i in zip(truth, guess):
            self.add(truth_i, guess_i)

class Trainer:
    def __init__(self, optimizer: torch.optim.Optimizer):
        self.optimizer = optimizer

    def run(self, model, labels, train, loss, batch_size): 
        model.train()       
        train_loader = DataLoader(train, batch_size=batch_size, shuffle=False)

        cm = ConfusionMatrix(labels)

        for batch in train_loader:
            loss_value, y_pred, y_actual = self.update(model, loss, batch)
            _, best = y_pred.max(1)
            yt = y_actual.cpu().int().numpy()
            yp = best.cpu().int().numpy()
            cm.add_batch(yt, yp)

        print(cm.get_all_metrics())
        return cm
    
    def update(self, model, loss, batch):
        self.optimizer.zero_grad()
        x, lengths, y = batch
        lengths, perm_idx = lengths.sort(0, descending=True)
        x_sorted = x[perm_idx]
        y_sorted = y[perm_idx]
        y_sorted = y_sorted.to('cuda:0')
        inputs = (x_sorted.to('cuda:0'), lengths)
        y_pred = model(inputs)
        loss_value = loss(y_pred, y_sorted)
        loss_value.backward()
        self.optimizer.step()
        return loss_value.item(), y_pred, y_sorted

class Evaluator:
    def __init__(self):
        pass

    def run(self, model, labels, dataset, batch_size=1):
        model.eval()
        valid_loader = DataLoader(dataset, batch_size=batch_size)
        cm = ConfusionMatrix(labels)
        for batch in valid_loader:
            y_pred, y_actual = self.inference(model, batch)
            _, best = y_pred.max(1)
            yt = y_actual.cpu().int().numpy()
            yp = best.cpu().int().numpy()
            cm.add_batch(yt, yp)
        return cm

    def inference(self, model, batch):
        with torch.no_grad():
            x, lengths, y = batch
            lengths, perm_idx = lengths.sort(0, descending=True)
            x_sorted = x[perm_idx]
            y_sorted = y[perm_idx]
            y_sorted = y_sorted.to('cuda:0')
            inputs = (x_sorted.to('cuda:0'), lengths)
            y_pred = model(inputs)
            return y_pred, y_sorted

def fit(model, labels, optimizer, loss, epochs, batch_size, train, valid, test):

    trainer = Trainer(optimizer)
    evaluator = Evaluator()
    best_acc = 0.0
    best_f1 = 0.0
    
    for epoch in range(epochs):
        print('EPOCH {}'.format(epoch + 1))
        print('=================================')
        print('Training Results')
        cm = trainer.run(model, labels, train, loss, batch_size)
        print('Validation Results')
        cm = evaluator.run(model, labels, valid)
        print(cm.get_all_metrics())

        # if cm.get_f() > best_f1:
        #     print('New best model {:.2f}'.format(cm.get_f()))
        #     best_f1 = cm.get_f()
        #     torch.save(model.state_dict(), f'/root/autodl-tmp/finetuned_models/{MODEL_NAME}_{DATASET_NAME}_code_checkpoint.pth')
        
        if cm.get_acc() > best_acc:
            print('New best model {:.2f}'.format(cm.get_acc()))
            best_acc = cm.get_acc()
            torch.save(model.state_dict(), f'/root/autodl-tmp/finetuned_models/{MODEL_NAME}_{DATASET_NAME}_code_checkpoint.pth')

    if test:
        model.load_state_dict(torch.load(f'/root/autodl-tmp/finetuned_models/{MODEL_NAME}_{DATASET_NAME}_code_checkpoint.pth'))
        cm = evaluator.run(model, labels, test)
        print('Final result')
        print(cm.get_all_metrics())
    return cm.get_acc()

def whitespace_tokenizer(words: str) -> List[str]:
    return words.split() 

def sst2_tokenizer(words: str) -> List[str]:
    REPLACE = { "'s": " 's ",
                "'ve": " 've ",
                "n't": " n't ",
                "'re": " 're ",
                "'d": " 'd ",
                "'ll": " 'll ",
                ",": " , ",
                "!": " ! ",
                }
    words = words.lower()
    words = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", words)
    for k, v in REPLACE.items():
            words = words.replace(k, v)
    return [w.strip() for w in words.split()]


class Reader:

    def __init__(self, files, lowercase=True, min_freq=0,
                 tokenizer=sst2_tokenizer, vectorizer=None):
        self.lowercase = lowercase
        self.tokenizer = tokenizer
        build_vocab = vectorizer is None
        self.vectorizer = vectorizer if vectorizer else self._vectorizer
        x = Counter()
        y = Counter()
        for file_name in files:
            if file_name is None:
                continue
            with codecs.open(file_name, encoding='utf-8', mode='r') as f:
                for line in f:
                    words = line.split()
                    y.update(words[0])

                    if build_vocab:
                        words = self.tokenizer(' '.join(words[1:]))
                        words = words if not self.lowercase else [w.lower() for w in words]
                        x.update(words)
        self.labels = list(y.keys())

        if build_vocab:
            x = dict(filter(lambda cnt: cnt[1] >= min_freq, x.items()))
            alpha = list(x.keys())
            alpha.sort()
            self.vocab = {w: i+1 for i, w in enumerate(alpha)}
            self.vocab['[PAD]'] = 0

        self.labels.sort()

    def _vectorizer(self, words: List[str]) -> List[int]:
        return [self.vocab.get(w, 0) for w in words]

    def load(self, filename: str) -> TensorDataset:
        label2index = {l: i for i, l in enumerate(self.labels)}
        xs = []
        lengths = []
        ys = []
        with codecs.open(filename, encoding='utf-8', mode='r') as f:
            for line in f:
                words = line.split()
                ys.append(label2index[words[0]])
                words = self.tokenizer(' '.join(words[1:]))
                words = words if not self.lowercase else [w.lower() for w in words]
                vec = self.vectorizer(words)
                lengths.append(len(vec))
                xs.append(torch.tensor(vec, dtype=torch.long))
        x_tensor = torch.nn.utils.rnn.pad_sequence(xs, batch_first=True)
        lengths_tensor = torch.tensor(lengths, dtype=torch.long)
        y_tensor = torch.tensor(ys, dtype=torch.long)
        return TensorDataset(x_tensor, lengths_tensor, y_tensor)

def init_embeddings(vocab_size, embed_dim, unif):
    return np.random.uniform(-unif, unif, (vocab_size, embed_dim))
    

class EmbeddingsReader:

    @staticmethod
    def from_text(filename, vocab, unif=0.25):
        
        with io.open(filename, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                line = line.rstrip("\n ")
                values = line.split(" ")

                if i == 0:
                    # fastText style
                    if len(values) == 2:
                        weight = init_embeddings(len(vocab), values[1], unif)
                        continue
                    # glove style
                    else:
                        weight = init_embeddings(len(vocab), len(values[1:]), unif)
                word = values[0]
                if word in vocab:
                    vec = np.asarray(values[1:], dtype=np.float32)
                    weight[vocab[word]] = vec
        if '[PAD]' in vocab:
            weight[vocab['[PAD]']] = 0.0
        
        embeddings = nn.Embedding(weight.shape[0], weight.shape[1])
        embeddings.weight = nn.Parameter(torch.from_numpy(weight).float())
        return embeddings, weight.shape[1]
    
    @staticmethod
    def from_binary(filename, vocab, unif=0.25):
        def read_word(f):

            s = bytearray()
            ch = f.read(1)

            while ch != b' ':
                s.extend(ch)
                ch = f.read(1)
            s = s.decode('utf-8')
            # Only strip out normal space and \n not other spaces which are words.
            return s.strip(' \n')

        vocab_size = len(vocab)
        with io.open(filename, "rb") as f:
            header = f.readline()
            file_vocab_size, embed_dim = map(int, header.split())
            weight = init_embeddings(len(vocab), embed_dim, unif)
            if '[PAD]' in vocab:
                weight[vocab['[PAD]']] = 0.0
            width = 4 * embed_dim
            for i in range(file_vocab_size):
                word = read_word(f)
                raw = f.read(width)
                if word in vocab:
                    vec = np.fromstring(raw, dtype=np.float32)
                    weight[vocab[word]] = vec
        embeddings = nn.Embedding(weight.shape[0], weight.shape[1])
        embeddings.weight = nn.Parameter(torch.from_numpy(weight).float())
        return embeddings, embed_dim



# Run

In [6]:
def convert_json_to_txt(in_file_name, out_file_name):
    df_data = pd.read_json(in_file_name)
    for _, row in df_data.iterrows():
        label = row['label']
        commit_patch = row['commit_patch']
        content = f'{int(label)} {commit_patch}\n'
        write_to_file(content, out_file_name)

in_train_file = f'/root/autodl-tmp/output_dataset_1/{DATASET_NAME}/train.json'      
in_val_file = f'/root/autodl-tmp/output_dataset_1/{DATASET_NAME}/val.json'      
in_test_file = f'/root/autodl-tmp/output_dataset_1/{DATASET_NAME}/test.json'  

out_train_file = f'/root/autodl-tmp/output_dataset_1/{DATASET_NAME}/train_code.txt'      
out_val_file = f'/root/autodl-tmp/output_dataset_1/{DATASET_NAME}/val_code.txt'      
out_test_file = f'/root/autodl-tmp/output_dataset_1/{DATASET_NAME}/test_code.txt'

remove_file_if_exist(out_train_file)
remove_file_if_exist(out_val_file)
remove_file_if_exist(out_test_file)

convert_json_to_txt(in_train_file, out_train_file)
convert_json_to_txt(in_val_file, out_val_file)
convert_json_to_txt(in_val_file, out_test_file)


In [7]:
BASE = f'/root/autodl-tmp/output_dataset_1/{DATASET_NAME}'
TRAIN = os.path.join(BASE, 'train_code.txt')
VALID = os.path.join(BASE, 'val_code.txt')
TEST = os.path.join(BASE, 'test_code.txt')
PRETRAINED_EMBEDDINGS_FILE = '/root/autodl-tmp/GoogleNews-vectors-negative300.bin'

r = Reader((TRAIN, VALID, TEST,))
train = r.load(TRAIN)
valid = r.load(VALID)
test = r.load(TEST)

embed_dim = 300
embeddings = nn.Embedding(len(r.vocab), embed_dim)
# embeddings, embed_dim = EmbeddingsReader.from_binary(PRETRAINED_EMBEDDINGS_FILE, r.vocab)
model  = ConvClassifier(embeddings, len(r.labels), embed_dim)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters") 

model.to('cuda:0')
loss = torch.nn.NLLLoss()
loss = loss.to('cuda:0')

learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adadelta(learnable_params, lr=1.0)

fit(model, r.labels, optimizer, loss, 10, 50, train, valid, test)


Model has 30039302 parameters
EPOCH 1
Training Results


/tmp/ipykernel_607/579723216.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._cm = np.zeros((nc, nc), dtype=np.int)


{'acc': 0.5295306534920279, 'precision': 0.43238095238095237, 'recall': 0.43027349038721907, 'f1': 0.4313246471226927}
Validation Results
{'acc': 0.5853822835971708, 'precision': 0.0, 'recall': 0.0, 'f1': 0}
New best model 0.59
EPOCH 2
Training Results
{'acc': 0.5211093644733887, 'precision': 0.4225352112676056, 'recall': 0.42242079610073113, 'f1': 0.42247799593771157}
Validation Results
{'acc': 0.5863927248231728, 'precision': 0.8, 'recall': 0.003249390739236393, 'f1': 0.006472491909385114}
New best model 0.59
EPOCH 3
Training Results
{'acc': 0.5288569503705367, 'precision': 0.42925457102672293, 'recall': 0.41321418900622797, 'f1': 0.42108167770419425}
Validation Results
{'acc': 0.5870663523071741, 'precision': 0.7272727272727273, 'recall': 0.006498781478472786, 'f1': 0.012882447665056359}
New best model 0.59
EPOCH 4
Training Results
{'acc': 0.5390747810464855, 'precision': 0.43997668997669, 'recall': 0.40888166802057946, 'f1': 0.423859649122807}
Validation Results
{'acc': 0.587403166

0.6089592455372179

# Custom Reliability Validation

In [8]:
# epoch 8:
# {'acc': 0.6228473019517795, 'precision': 0.6200274348422496, 'recall': 0.3035594358629953, 'f1': 0.4075743913435527}
# {'acc': 0.5640068886337543, 'precision': 0.4943353474320242, 'recall': 0.8791134989926125, 'f1': 0.6328257191201353}


from sklearn import metrics
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('using', device)
print('len(r.vocab):', len(r.vocab))
print('r.labels:', r.labels)

BASE = f'/root/autodl-tmp/output_dataset_1/{DATASET_NAME}'
TRAIN = os.path.join(BASE, 'train_code.txt')
VALID = os.path.join(BASE, 'val_code.txt')
TEST = os.path.join(BASE, 'test_code.txt')

r = Reader((TRAIN, VALID, TEST,))
train = r.load(TRAIN)
valid = r.load(VALID)
test = r.load(TEST)

embed_dim = 300
embeddings = nn.Embedding(len(r.vocab), embed_dim)
model  = ConvClassifier(embeddings, len(r.labels), embed_dim)
model.load_state_dict(torch.load(f'/root/autodl-tmp/finetuned_models/{MODEL_NAME}_{DATASET_NAME}_code_checkpoint.pth'))
model.to(device)
model.eval()

total_acc_test = 0
predict_all = np.array([], dtype=int)
labels_all = np.array([], dtype=int)
valid_loader = DataLoader(valid, batch_size=1)

for batch in tqdm(valid_loader):
    with torch.no_grad():
        x, lengths, y = batch
        x = x.to(device)
        y = y.to(device)
        y_pred = model((x, lengths))
        acc = (y_pred.argmax(dim=1) == y).sum().item()
        total_acc_test += acc
        
        y = y.data.cpu().numpy()
        predic = y_pred.argmax(dim=1).data.cpu().numpy()
        labels_all = np.append(labels_all, y)
        predict_all = np.append(predict_all, predic)

report = metrics.classification_report(labels_all, predict_all, target_names=['benign', 'vulnerable'], digits=4)
confusion = metrics.confusion_matrix(labels_all, predict_all)
# print(f'Test Accuracy: {total_acc_test / len(valid): .3f}')
print(report)
print(confusion)


using cuda
len(r.vocab): 99228
r.labels: ['0', '1']


100%|██████████| 2969/2969 [00:04<00:00, 626.14it/s]

              precision    recall  f1-score   support

      benign     0.6106    0.9166    0.7329      1738
  vulnerable     0.5972    0.1747    0.2703      1231

    accuracy                         0.6090      2969
   macro avg     0.6039    0.5456    0.5016      2969
weighted avg     0.6050    0.6090    0.5411      2969

[[1593  145]
 [1016  215]]
